###
- KOI 중등부 전설의 문제. 당시 만점자가 없었다고 한다.

### 아이디어
- 기존에 풀었던 스위핑 세그 문제처럼 한 축을 잡고 스위핑하면서 푼다고 했을 때, 곤란해지는 지점은 y축이나 x축 위에 여러개의 광산이 있을때이다. 
- 광산의 개수가 3000개로, 구분 가능한 x, y좌표는 6000개만 나온다. \
만약 가능한 x, y좌표에 대해 모든 쌍을 구한다면 $6000^2$ 개 정도가 나오는데, 만약 쿼리를 구하는 것이 $\log N$ 정도라면 충분히 구할 수 있을 것이다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from dataclasses import dataclass
@dataclass
class Node :
  ls: int = 0
  rs: int = 0
  ms: int = 0
  s: int = 0

class ST : 
  def __init__(self, size): #default는 Node()이다.
    self._size = _size = 1 << (size - 1).bit_length()
    self.seg = [Node() for _ in range(2 * _size)]

  def __getitem__(self, i):
    return self.seg[i + self._size]

  def update(self, i, v) :
    i += self._size
    self.seg[i].ms += v
    self.seg[i].ls = self.seg[i].rs = self.seg[i].s = self.seg[i].ms
    while i > 1 :
      i >>= 1
      self.seg[i] = self.merge(self.seg[i*2], self.seg[i*2+1])

  def merge(self, l: Node, r: Node):
    ret = Node()
    ret.s = l.s + r.s
    ret.ls = max(l.ls, l.s + r.ls)
    ret.rs = max(r.rs, r.s + l.rs)
    ret.ms = max(ret.s, l.ms, r.ms, l.rs + r.ls)
    return ret

def sol() :
  N = int(input())
  X = set()
  Y = set()
  T = []
  for _ in range(N) :
    x, y, w = map(int, input().split())
    T.append([x, y, w])
    X.add(x)
    Y.add(y)
  
  X = {x: i for i, x in enumerate(sorted(X))}
  Y = {y: i for i, y in enumerate(sorted(Y))}
  T = [[X[x], Y[y], w] for x, y, w in T] #compression
  T.sort(key=lambda x: (x[1], x[0]))

  ans = 0
  for i in range(N) : #가능한 모든 조합(for i in range(N) : for j in range(i, N))
    if i and T[i][1] == T[i-1][1] : continue #p1은 같은 y좌표 중 가장 왼쪽에 있는 한개만 처리해야 한다.
    st = ST(N)
    for j in range(i, N) :
      st.update(T[j][0], T[j][2])
      if j == N - 1 or T[j][1] != T[j+1][1] : #p2는 같은 y좌표 중 가장 오른쪽에 있는 한개만 처리해야 한다.
        ans = max(ans, st.seg[1].ms)  
  print(ans)

sol()

7
2 8 2
5 5 3
3 3 -1
10 2 5
9 7 -2
6 7 -1
7 3 -1
10
4 9 2
6 10 -1
6 8 3
5 6 5
8 5 10
7 6 -7
9 10 4
20 1 1
10 8 6
10 6 -5
4
2 2 4
1 2 6
2 1 7
1 1 -1000
4
1 1 4
1 2 6
2 1 7
2 2 -1000

### 풀이
- 가능한 x, y 좌표 쌍에 대해 모두 시도해보는 아이디어는 맞았다.
- 16993(연속합과 쿼리)에서 사용된 최대 연속누적합 세그트리의 아이디어를 사용하여 구역 $p_1, p_2$ 에 대한 개발 이익을 $\Omicron(\log N)$ 에 구할 수 있다.
- $y$ 기준으로 좌표를 정렬한 뒤, $x$ 기준으로 스위핑하면서 구간 $[p_1, p_2]$ 에 대한 개발 이익을 구한다. 그 과정에서 쿼리의 최댓값이 정답이다.
- $w_i > 0$ 인 금광이 적어도 하나는 존재하므로 누적합 노드의 최솟값은 0으로 충분하다.
- 같은 y좌표를 가진 금광에 대해서, 가장 $p_1$ 은 x좌표가 작은 금광 한개만 처리해야 한다.
  - 2 2 4 / 1 2 6 / 2 1 7 / 1 1 -1000 처럼 직사각형 모양으로 금광이 있는 경우, x좌표가 더 높은 곳에서 중복처리할 때 for j (i, N) 에 의해 x좌표가 더 낮은 금광이 계산에서 누락되게 된다.
- 마찬가지로 최대 구간합을 구하는 과정에서 $p_2$ 도 x좌표가 가장 큰 금광 한개를 잡고 처리해야 한다.